## 7.4　Convolution／Pooling レイヤの実装

In [1]:
import os
import sys
import numpy as np

In [2]:
os.chdir('/Users/yuta.shimizu/Downloads/Machine Learning/deep-learning-from-scratch-master/ch01')
sys.path.append(os.pardir)

In [3]:
from common.util import im2col

### 7.4.1　4 次元配列

In [4]:
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [5]:
print(x[0].shape)
print(x[1].shape)

(1, 28, 28)
(1, 28, 28)


In [6]:
x[0, 0][:1]

array([[0.950543  , 0.34356156, 0.41525097, 0.8591098 , 0.9332909 ,
        0.72716431, 0.89415206, 0.15345353, 0.76525538, 0.16490336,
        0.76964274, 0.70954623, 0.8368931 , 0.55177753, 0.90738241,
        0.00792462, 0.76818761, 0.50721768, 0.57905534, 0.57349874,
        0.33806141, 0.5550905 , 0.75851536, 0.96316926, 0.80017213,
        0.64085273, 0.02378796, 0.72870626]])

### 7.4.3　Convolution レイヤの実装

In [7]:
x1 = np.random.rand(1,3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


In [8]:
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


In [9]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w =  int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

`out_h` と `out_w` は、P. 212 の式を利用している。<br>
また、バッチ処理になってもフィルターの次元は変化しない。（P. 78 参照）

### 7.4.4　Pooling レイヤの実装

In [10]:
class pooling:
    def __init__(self, pool_h, pool_w, stride=2, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        out = np.max(col, axis=1)
        out = reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out